In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import requests
import xarray as xr


In [2]:
def getL3Burl(instr_name, prod_suff, temp_res, target_dt):
    from datetime import timedelta
    satfileurl = ''
    if 'MERIS' in instr_name:
        if 'DAY' in temp_res:
            l3filename = target_dt.strftime('M%Y%j.L3b_DAY_CYAN.nc')
        elif '7D' in temp_res:
            idyjul = int(target_dt.strftime('%j')) - 1
            idyjul_hi = 7 * (1 + int(idyjul / 7)) #calculate the next number divisible by 7
            dt_hi = target_dt + timedelta(days=idyjul_hi-idyjul-1)
            dt_lo = dt_hi - timedelta(days=6)
            l3filename = dt_lo.strftime('M%Y%j') + dt_hi.strftime('%Y%j.L3b_7D_'+ prod_suff + '.nc') #MERIS 7-day L3b
        else:
            print('ERROR - Bad temporal specification:',temp_res)
            return ''
    elif 'OLCI' in instr_name:
        if 'DAY' in temp_res:
            l3filename = target_dt.strftime('L%Y%j.L3b_DAY_CYAN.nc')
            #l3filename = target_dt.strftime('L%Y%j.L3m_DAY_CYAN_CI_cyano_CYAN_CONUS_300m.tif') #OLCI daily L3b
        elif '7D' in temp_res: #L20170152017021.L3b_7D_S3A_CYAN.nc
            idyjul = int(target_dt.strftime('%j')) - 1
            idyjul_hi = 7 * (1 + int(idyjul / 7)) #calculate the next number divisible by 7
            dt_hi = target_dt + timedelta(days=idyjul_hi-idyjul-1)
            dt_lo = dt_hi - timedelta(days=6)
            l3filename = dt_lo.strftime('L%Y%j') + dt_hi.strftime('%Y%j.L3b_7D_S3A_' + prod_suff + '.nc') 
            #MERIS 7-day L3b
        else:
            print('ERROR - Bad temporal specification:',temp_res)
            return ''
    else:
        print('ERROR - Bad instrument specification:',instr_name)
        return ''

    return 'https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/' + l3filename


def load_L3Burl(url, filecache=False, cache_dir='./data/L3B'):
    '''
    Load or download (filecache=True) an L3B file given URL
    '''
    from netrc import netrc
    from aiohttp import BasicAuth
    import fsspec
    
    usr, _, pwd = netrc().authenticators("urs.earthdata.nasa.gov")
    auth = BasicAuth(usr, pwd)
    
    if filecache:
        fs = fsspec.filesystem(
            "filecache",
            cache_storage=cache_dir,
            target_protocol="https",
            target_options={"client_kwargs": {"auth": auth}},
        )
        
    else:
        fs = fsspec.filesystem('simplecache',
            target_protocol="https",
            target_options={"client_kwargs": {"auth": auth}},
        )

    ds = xr.open_dataset(fs.open(url), group="level-3_binned_data")
        
    return ds


def process_L3B_file(ds):
    '''
    Process L3b_DAY_CYAN nc file into pandas dataframe, convert bins to lat/lon location
    '''
    df_BinIndex = pd.DataFrame(ds.BinIndex.values).reset_index()

    df = pd.DataFrame(ds.BinList.values)
    for k in list(ds.keys())[1:-1]:
        tmp = pd.DataFrame(ds[k].values)
        df[k] = tmp['sum']

    out = pd.merge_asof(left=df, right=df_BinIndex, left_on="bin_num", right_on="start_num", direction='backward')

    nrows = ds.sizes['binIndexDim']
    latbin = (np.arange(0, nrows, dtype=np.float_) + 0.5) * (180.0 / nrows) - 90.0
    out['clat'] = out['index'].apply(lambda x: latbin[x])
    out['clon'] = (360.0 * (out['bin_num'] - out['start_num'] + 0.5) / out['max']) - 180.0
    out['north'] = out['clat'] + (90.0 / nrows)
    out['south'] = out['clat'] - (90.0 / nrows)
    out['west'] = out['clon'] - (180.0 / out['max'])
    out['east'] = out['clon'] + (180.0 / out['max'])
    
    out_cols = ['bin_num', 'CI_stumpf','CI_cyano', 'CI_noncyano', 'MCI_stumpf', 
                'clat','clon', 'north', 'south', 'west', 'east']
    
    return out[out_cols]


## Download daily data for April 1 - 30, 2024

In [3]:
def convert_int_to_datetime_manual(date_int):
    year = date_int // 10000
    month = (date_int % 10000) // 100
    day = date_int % 100
    return datetime.date(year, month, day)

day_first = convert_int_to_datetime_manual(20240401)
day_last = convert_int_to_datetime_manual(20240430)
(day_last - day_first).days

29

In [4]:
%%time

local = "./data/L3B/"
df = pd.DataFrame()

for i in range((day_last - day_first).days+1):
    
    day_of = day_first + datetime.timedelta(days=i)
    url = getL3Burl(instr_name='OLCI', prod_suff='CYAN', temp_res='DAY', target_dt=day_of)
    fn = url.split('/')[-1]
    print(day_of, ": Processing", url)
    try:
        r = requests.get(url)
        with open(local + fn, 'wb') as f:
            f.write(r.content)

        ds = xr.open_dataset(local + fn, group="level-3_binned_data")
        
        df_tmp = process_L3B_file(ds=ds)
        df_tmp['date'] = day_of
        df = pd.concat([df, df_tmp], axis=0)
        print("  Complete: ", df_tmp.shape)
        
        os.remove(local + fn)
        
    except:
        print("No data for", day_of)
        
print(df.shape)

2024-04-01 : Processing https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/L2024092.L3b_DAY_CYAN.nc
  Complete:  (6690548, 12)
2024-04-02 : Processing https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/L2024093.L3b_DAY_CYAN.nc
  Complete:  (6255716, 12)
2024-04-03 : Processing https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/L2024094.L3b_DAY_CYAN.nc
  Complete:  (6106833, 12)
2024-04-04 : Processing https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/L2024095.L3b_DAY_CYAN.nc
  Complete:  (6117719, 12)
2024-04-05 : Processing https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/L2024096.L3b_DAY_CYAN.nc
  Complete:  (9454355, 12)
2024-04-06 : Processing https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/L2024097.L3b_DAY_CYAN.nc
  Complete:  (9060560, 12)
2024-04-07 : Processing https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/L2024098.L3b_DAY_CYAN.nc
  Complete:  (8532687, 12)
2024-04-08 : Processing https://oceandata.sci.gsfc.nasa.gov/cgi/getfile/L2024099.L3b_DAY_CYAN.nc
  Complete:  (6702112, 12)
2024-04-

In [5]:
%%time
# Jordan Lake
targ_lat = [35.676263, 35.885728]
targ_lon = [-79.07959, -78.95462]

df_jordan = df.loc[(df.west>=np.min(targ_lon)) &
                    (df.east<=np.max(targ_lon)) &
                    (df.south>=np.min(targ_lat)) &
                    (df.north<=np.max(targ_lat))]

df_jordan.shape

CPU times: user 2.64 s, sys: 9.89 s, total: 12.5 s
Wall time: 15.7 s


(3913, 12)

In [6]:
%%time
# Lake Mendota
max_deg = 0.1
lat = 43.175
lon = -89.465
north = lat + max_deg
south = lat - max_deg
east = lon + max_deg
west = lon - max_deg
targ_lat = [south, north]
targ_lon = [west, east]

df_mendota = df.loc[(df.west>=np.min(targ_lon)) &
                    (df.east<=np.max(targ_lon)) &
                    (df.south>=np.min(targ_lat)) &
                    (df.north<=np.max(targ_lat))]

df_mendota.shape

CPU times: user 1.75 s, sys: 1.08 s, total: 2.83 s
Wall time: 3 s


(4252, 12)

In [7]:
%%time
# Lake Mattamuskeet
max_deg = 0.1
lat = 35.5
lon = -76.2
north = lat + max_deg
south = lat - max_deg
east = lon + max_deg
west = lon - max_deg
targ_lat = [south, north]
targ_lon = [west, east]

df_matt = df.loc[(df.west>=np.min(targ_lon)) &
                    (df.east<=np.max(targ_lon)) &
                    (df.south>=np.min(targ_lat)) &
                    (df.north<=np.max(targ_lat))]

df_matt.shape

CPU times: user 1.59 s, sys: 1.1 s, total: 2.69 s
Wall time: 2.85 s


(29196, 12)

In [8]:
%%time
path1 = './data/L3B/L3B_CYAN_DAILY_JORDAN.parquet'
path2 = './data/L3B/L3B_CYAN_DAILY_MENDOTA.parquet'
path3 = './data/L3B/L3B_CYAN_DAILY_MATT.parquet'

d1 = pd.read_parquet(path1)
d2 = pd.read_parquet(path2)
d3 = pd.read_parquet(path3)
print('Jordan:', d1.shape)
print('Mendota:', d2.shape)
print('Mattamuskeet:', d3.shape)

Jordan: (332990, 12)
Mendota: (236013, 12)
Mattamuskeet: (1811653, 12)
CPU times: user 460 ms, sys: 553 ms, total: 1.01 s
Wall time: 1.7 s


In [9]:
%%time
d1 = pd.concat([d1, df_jordan], axis=0)
d2 = pd.concat([d2, df_mendota], axis=0)
d3 = pd.concat([d3, df_matt], axis=0)
print('Jordan:', d1.shape)
print('Mendota:', d2.shape)
print('Mattamuskeet:', d3.shape)

Jordan: (336903, 12)
Mendota: (240265, 12)
Mattamuskeet: (1840849, 12)
CPU times: user 89.7 ms, sys: 94.3 ms, total: 184 ms
Wall time: 252 ms


In [10]:
%%time
d1.to_parquet(path1)
d2.to_parquet(path2)
d3.to_parquet(path3)

CPU times: user 909 ms, sys: 74.3 ms, total: 984 ms
Wall time: 921 ms
